In [1]:
from utils import get_gemini_api_key
GEMINI_API_KEY = get_gemini_api_key()
#print(OPENAI_API_KEY)
llm_config = {"model": "gemini-2.5-flash","api_key":GEMINI_API_KEY,"api_type":"google"}

# Task

In [2]:
task = '''
        Write a concise but engaging blogpost about
      Hope AI . Make sure the blogpost is
       within 100 words.
       '''

In [3]:
# creating a writer agent

In [4]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [5]:
reply = writer.generate_reply(messages=[{"content": task,"role": "user"}])

In [6]:
reply

{'content': "## The Dawn of Hope AI\n\nImagine AI not just optimizing, but uplifting. That's Hope AI. Beyond efficiency, it's about leveraging artificial intelligence for human flourishing. Think AI accelerating medical breakthroughs, creating sustainable solutions, or fostering genuine connection. It's the intelligent pursuit of a better world, empowering us to solve humanity's greatest challenges with compassion and innovation at its core. Hope AI isn't science fiction; it's the intelligent path to a brighter future, built on shared purpose.",
 'refusal': None,
 'role': 'assistant',
 'annotations': None,
 'audio': None,
 'function_call': None,
 'tool_calls': None}

In [7]:
# Create a critic agent to reflect on the work of the writer agent.

In [8]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of the writer and only provide constructive feedback to help improve the quality of the content.",
)

In [9]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg")

Critic (to Writer):


        Write a concise but engaging blogpost about
      Hope AI . Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Hope AI: A Brighter Tomorrow, Powered by Innovation**

Imagine artificial intelligence not as a distant threat, but as a beacon of progress. That’s Hope AI. It’s the promise of leveraging advanced technology to solve humanity's most pressing issues – from climate change and disease to poverty and educational gaps. Hope AI inspires optimism, transforming challenges into opportunities for groundbreaking solutions. It's about ethical, purpose-driven innovation that empowers communities, fosters global collaboration, and builds a future where technology uplifts every life. This isn't just AI; it's our collective potential, amplified for good.

--------------------------------------------------------------------------------
Critic (to Writer

# Nested chat

In [10]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO_Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [11]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal_Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [12]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics_Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",)

In [13]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta_Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

In [14]:
# Orchestrate the nested chats to solve the task

In [15]:
#This function is called inside the conversation and it takes in the writer's response as the input message to give its review

def reflection_message(recipient, messages, sender, config):
    content = recipient.chat_messages_for_summary(sender)[-1]['content']
    return f"Review the following content:\n\n {content}"

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    
    {
     "recipient": legal_reviewer,
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    
    {
     "recipient": ethics_reviewer,
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
    
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [16]:
critic.register_nested_chats(review_chats,trigger=writer)
# The trigger is being set like whenever the critic gets a message from writer the nested chat sequence should be triggered

In [17]:
res = critic.initiate_chat(
                           recipient=writer,
                           message=task,
                           max_turns=2,
                           summary_method="last_msg"
                           )

Critic (to Writer):


        Write a concise but engaging blogpost about
      Hope AI . Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Hope AI: Crafting a Brighter Future**

Imagine AI not just calculating, but caring. Hope AI embodies this vision: an intelligence driven by a singular purpose – to uplift humanity. It's the AI that accelerates cures for disease, designs sustainable solutions for our planet, and connects communities worldwide. More than just smart, it's empathetic, built to empower and inspire. Hope AI transforms challenges into opportunities, proving that technology can be a powerful force for good, a true beacon of progress for all.

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

**********************************

C:\anaconda3\envs\agenticai\Lib\site-packages\autogen\agentchat\conversable_agent.py:2192: UserWarning: Cannot extract summary using reflection_with_llm: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 5, model: gemini-2.5-flash\nPlease retry in 28.729474904s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_reques

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 5, model: gemini-2.5-flash\nPlease retry in 26.918281617s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.5-flash'

In [ ]:
SEO_reviewer.chat_messages_for_summary(critic)[-1]

In [ ]:
writer.chat_messages_for_summary(critic)

In [ ]:
legal_reviewer.chat_messages_for_summary(critic)

In [19]:
import pprint
pprint.pprint(res)

ChatResult(chat_id=26353796792345428501795835658818444836,
           chat_history=[{'content': '\n'
                                     '        Write a concise but engaging '
                                     'blogpost about\n'
                                     '      Hope AI . Make sure the blogpost '
                                     'is\n'
                                     '       within 100 words.\n'
                                     '       ',
                          'name': 'Critic',
                          'role': 'assistant'},
                         {'content': '**Hope AI: Crafting a Brighter Future**\n'
                                     '\n'
                                     'Imagine AI not just calculating, but '
                                     'caring. Hope AI embodies this vision: an '
                                     'intelligence driven by a singular '
                                     "purpose – to uplift humanity. It's the "
   